In [26]:
X_train = open('src-dev.txt').read().split('\n')
X_test = open('src-test.txt').read().split('\n')
y_train = open('tgt-dev.txt').read().split('\n')
y_test = open('tgt-test.txt').read().split('\n')

In [27]:
from torch import Tensor
tokens = set([])
for sentence in X_train+X_test:
    for word in sentence.split(' '):
        tokens.add(word)
tokens = list(tokens)
tokens.append('<sos>')
tokens.append('<eos>')
num_tokens = len(tokens)
print(num_tokens)

tag_list = set([])
for sentence in y_train+y_test:
    for tag in sentence.split(' '):
        tag_list.add(tag)
tag_list = list(tag_list)
num_tags = len(tag_list)
tag_list.sort()    
print(tag_list)

def get_one_hot(word):
    one_hot = torch.zeros(num_tokens)
    one_hot[tokens.index(word)] = 1
    return one_hot

def get_tag(tag):
    return tag_list.index(tag)

8522
['', 'adv', 'ccof', 'fragof', 'jjmod', 'jjmod__intf', 'k1', 'k1s', 'k1u', 'k2', 'k2g', 'k2p', 'k2s', 'k2u', 'k3', 'k4', 'k4a', 'k5', 'k7', 'k7a', 'k7p', 'k7t', 'k7tu', 'k7u', 'lwg__neg', 'lwg__psp', 'lwg__rp', 'lwg__vaux', 'lwg__vaux_cont', 'main', 'mod', 'mod__wq', 'nmod', 'nmod__adj', 'nmod__emph', 'nmod__k1inv', 'nmod__k2inv', 'nmod__pofinv', 'nmod__relc', 'pof', 'pof__cn', 'pof__inv', 'pof__redup', 'r6', 'r6-k1', 'r6-k2', 'r6v', 'rad', 'ras-k1', 'ras-k2', 'ras-k7', 'ras-k7p', 'ras-neg', 'ras-pof', 'rbmod', 'rbmod__relc', 'rd', 'rh', 'rs', 'rsp', 'rsym', 'rt', 'sent-adv', 'undef', 'vmod']


In [28]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import numpy as np

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_tokens, 1000)
        self.fc2 = nn.Linear(3 * num_tags, num_tags)
        self.fc3 = nn.Linear(1000,num_tags)
    
    def forward(self,x,x_1,x_2):
        x = Variable(x.T)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.relu(self.fc3(x))
        
        x_1 = Variable(x_1.T)
        x_1 = self.fc1(x_1)
        x_1 = F.relu(x_1)
        x_1 = F.relu(self.fc3(x_1))
        
        x_2 = Variable(x_2.T)
        x_2 = self.fc1(x_2)
        x_2 = F.relu(x_2)
        x_2 = F.relu(self.fc3(x_2))
        
        x_com = torch.cat((x, x_1, x_2), 0)
        final = F.softmax(self.fc2(x_com))
        return final
    
net = Net()
net = net.float()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)


Net(
  (fc1): Linear(in_features=8522, out_features=1000, bias=True)
  (fc2): Linear(in_features=195, out_features=65, bias=True)
  (fc3): Linear(in_features=1000, out_features=65, bias=True)
)

In [29]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

# print(criterion(torch.tensor([1,0]),torch.tensor([0.9,0.1])))

In [30]:
for epoch in range(2):
    running_loss = 0.0
    for i in range(len(X_train)):
#         try:
        print(i, end=" ")
        optimizer.zero_grad()
        words = X_train[i].split(' ')
        words.append('<eos>')
        words.insert(0,'<sos>')
        tags = y_train[i].split(' ')
        a = []
        for j in range(1,len(words)-1):
            a.append(net(get_one_hot(words[j]),get_one_hot(words[j+1]),get_one_hot(words[j-1])))
        out = torch.stack(a)
        out.to(device)
        b = [get_tag(tag) for tag in tags]
        res = torch.tensor(b)
        res.to(device)
        loss = torch.tensor(0)
        loss.to(device)
        loss = criterion(out,res)

        print(running_loss)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
#         except:
#             continue
print('Finished Training')

0 

<ipython-input-28-e786d0ef8d57>:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final = F.softmax(self.fc2(x_com))


0.0
1 4.174663066864014
2 8.34933853149414
3 12.523647785186768
4 16.697897911071777
5 20.872007846832275
6 25.04591178894043
7 29.219743728637695
8 33.393431663513184
9 37.56716585159302
10 41.74021244049072
11 45.91273736953735
12 50.08498191833496
13 54.25674772262573
14 58.42870235443115
15 62.600220680236816
16 66.76928663253784
17 70.9396915435791
18 75.1061840057373
19 79.27040338516235
20 83.43827724456787
21 87.6024022102356
22 91.76014614105225
23 95.91446828842163
24 100.06299495697021
25 104.21117925643921
26 108.34566259384155
27 112.47920274734497
28 116.59256839752197
29 120.70593500137329
30 124.82315492630005
31 128.94185066223145
32 133.01951122283936
33 137.1529746055603
34 141.257652759552
35 145.32564687728882
36 149.37098360061646
37 153.4250931739807
38 157.48205995559692
39 161.45614433288574
40 165.47071027755737
41 169.5459771156311
42 173.5889310836792
43 177.5166780948639
44 181.45653009414673
45 185.53082466125488
46 189.4850835800171
47 193.6829662322998
4

373 1491.067949295044
374 1494.8426461219788
375 1498.4956583976746
376 1502.282870054245
377 1506.0941064357758
378 1509.8297019004822
379 1513.613520860672
380 1517.3910026550293
381 1521.1125338077545
382 1525.040599346161
383 1528.8643577098846
384 1532.930017709732
385 1536.6280827522278
386 1540.4559676647186
387 1544.2943150997162
388 1548.1151790618896
389 1551.9933037757874
390 1555.8466112613678
391 1559.670930147171
392 1563.519324541092
393 1567.3545091152191
394 1571.1094748973846
395 1574.865941286087
396 1578.568600177765
397 1582.4875040054321
398 1586.3155043125153
399 1590.2827229499817
400 1594.1189153194427
401 1597.8422780036926
402 1601.7587192058563
403 1605.5140092372894
404 1609.2176313400269
405 1613.0056388378143
406 1616.9410512447357
407 1620.7100551128387
408 1624.4696383476257
409 1628.334416627884
410 1632.1414108276367
411 1636.0051896572113
412 1639.9337892532349
413 1643.7335274219513
414 1647.602314710617
415 1651.387915611267
416 1655.2964746952057


736 2856.2664840221405
737 2860.1515963077545
738 2863.932569026947
739 2867.7142062187195
740 2871.3953511714935
741 2875.126039505005
742 2878.779056072235
743 2882.4876935482025
744 2886.2434566020966
745 2890.108631134033
746 2893.75390124321
747 2897.5326068401337
748 2901.2857224941254
749 2905.0436630249023
750 2908.710471868515
751 2912.43062376976
752 2916.1006247997284
753 2919.7113873958588
754 2923.3992524147034
755 2926.9776821136475
756 2930.754157781601
757 2934.4538292884827
758 2938.2390015125275
759 2941.93763422966
760 2945.6192593574524
761 2949.3063073158264
762 2953.08931517601
763 2956.8115684986115
764 2960.4550256729126
765 2964.1581716537476
766 2967.896783351898
767 2971.5780856609344
768 2975.2626056671143
769 2979.068465948105
770 2982.958916425705
771 2986.7188329696655
772 2990.541102409363
773 2994.266626596451
774 2998.0771565437317
775 3001.8629734516144
776 3005.63397359848
777 3009.33460688591
778 3013.0190727710724
779 3016.8007066249847
780 3020.54

...

72 266.43561816215515
73 270.1922171115875
74 273.80659341812134
75 277.43099904060364
76 281.21037697792053
77 284.66158628463745
78 288.2390146255493
79 291.9966080188751
80 295.72028517723083
81 299.5568382740021
82 303.0787556171417
83 306.7634062767029
84 310.31414127349854
85 314.0184783935547
86 317.8416142463684
87 321.4442193508148
88 325.09637665748596
89 328.9511694908142
90 332.725617647171
91 336.3992578983307
92 340.1989552974701
93 344.04537200927734
94 347.79480719566345
95 351.31259298324585
96 355.01417899131775
97 358.5081307888031
98 362.3685977458954
99 366.21754336357117
100 369.89821696281433
101 373.69196939468384
102 377.3999195098877
103 381.12867164611816
104 384.69277691841125
105 388.2263913154602
106 391.69924569129944
107 395.45264172554016
108 399.19465923309326
109 403.01850628852844
110 406.7690968513489
111 410.465172290802
112 414.40526723861694
113 418.0552318096161
114 421.7193953990936
115 425.4258863925934
116 429.1411099433899
117 432.6345582008

439 1622.0405666828156
440 1625.7374231815338
441 1629.4494738578796
442 1632.9947628974915
443 1636.785960674286
444 1640.4823524951935
445 1644.157877922058
446 1647.7236106395721
447 1651.281301498413
448 1655.0262293815613
449 1658.6172482967377
450 1662.3455772399902
451 1665.9183456897736
452 1669.5457696914673
453 1673.262751340866
454 1676.8977518081665
455 1680.5013766288757
456 1684.1173498630524
457 1687.7017064094543
458 1691.3456938266754
459 1695.086157798767
460 1698.7001566886902
461 1702.5653758049011
462 1706.2879693508148
463 1710.1200001239777
464 1713.8021438121796
465 1717.5907502174377
466 1721.1791894435883
467 1724.720838546753
468 1728.3961517810822
469 1732.0516202449799
470 1735.575607061386
471 1739.2086300849915
472 1742.934077501297
473 1746.7212626934052
474 1750.4732475280762
475 1754.1802380084991
476 1757.8317019939423
477 1761.4861767292023
478 1765.1434032917023
479 1768.6192305088043
480 1772.3482022285461
481 1776.0868315696716
482 1779.8012168407

802 2951.138243198395
803 2954.807736635208
804 2958.4659402370453
805 2962.1066834926605
806 2965.8590965270996
807 2969.574893474579
808 2973.203229904175
809 2976.8821291923523
810 2980.450611114502
811 2984.18879032135
812 2987.799065351486
813 2991.422666788101
814 2995.1293482780457
815 2998.801056623459
816 3002.470602750778
817 3006.2291853427887
818 3009.9008646011353
819 3013.7557661533356
820 3017.4088563919067
821 3021.21129322052
822 3024.9189755916595
823 3028.5927510261536
824 3032.297150850296
825 3035.951578140259
826 3039.6793093681335
827 3043.454831600189
828 3047.0801153182983
829 3050.7638351917267
830 3054.460342168808
831 3058.0773062705994
832 3061.764694213867
833 3065.3883907794952
834 3069.2624900341034
835 3072.8128695487976
836 3076.4530730247498
837 3080.0170645713806
838 3083.6266129016876
839 3087.0909168720245
840 3090.7674055099487
841 3094.41783952713
842 3097.9686431884766
843 3101.751986503601
844 3105.249151468277
845 3109.060643196106
846 3112.66

1161 4268.395951032639
1162 4272.07944560051
1163 4275.766877889633
1164 4279.50439119339
1165 4283.296524763107
1166 4286.981977701187
1167 4290.633465051651
1168 4294.324123620987
1169 4297.97876906395
1170 4301.652485847473
1171 4305.334645032883
1172 4308.979115247726
1173 4312.536646366119
1174 4316.165958881378
1175 4319.904017686844
1176 4323.436765432358
1177 4327.136183261871
1178 4330.615343809128
1179 4334.126301050186
1180 4337.883541107178
1181 4341.703222990036
1182 4345.363943576813
1183 4348.896896362305
1184 4352.715695619583
1185 4356.4131281375885
1186 4360.169333696365
1187 4363.760851621628
1188 4367.461275815964
1189 4371.2453989982605
1190 4374.945253610611
1191 4378.532902240753
1192 4382.217808246613
1193 4385.731947183609
1194 4389.390055656433
1195 4393.110884189606
1196 4396.875377416611
1197 4400.460634946823
1198 4404.049823999405
1199 4407.72038769722
1200 4411.532562494278
1201 4415.137237787247
1202 4418.799385070801
1203 4422.429949998856
1204 4426.027

1518 5576.379981040955
1519 5580.080914735794
1520 5583.641020059586
1521 5587.257522583008
1522 5590.874491453171
1523 5594.645157575607
1524 5598.316443681717
1525 5601.8250460624695
1526 5605.507525205612
1527 5609.2454998493195
1528 5612.706221103668
1529 5616.29797244072
1530 5619.947451353073
1531 5623.706834077835
1532 5627.272998332977
1533 5630.703781604767
1534 5634.341128826141
1535 5637.71807217598
1536 5641.226352930069
1537 5644.810821771622
1538 5648.474373102188
1539 5652.112530469894
1540 5655.806993722916
1541 5659.39631819725
1542 5663.007273435593
1543 5666.7378051280975
1544 5670.468859672546
1545 5674.168644666672
1546 5677.863512277603
1547 5681.563316106796
1548 5685.136559247971
1549 5688.734799623489
1550 5692.418566703796
1551 5696.120068311691
1552 5699.793565988541
1553 5703.429857969284
1554 5707.154327630997
1555 5710.854331731796
1556 5714.5419409275055
1557 5718.268855333328
1558 5721.951740026474
1559 5725.588935613632
1560 5729.243809223175
1561 5732.

In [36]:
def predict_tag(probs):
    values, indices = torch.max(probs,0)
    return tag_list[indices]

In [41]:
f = open('out.txt','w')
with torch.no_grad():
    for i in range(len(X_test)):
        words = X_test[i].split(' ')
        words.append('<eos>')
        words.insert(0,'<sos>')
        tags = y_test[i].split(' ')
        a = []
        for j in range(1,len(words)-1):
            x = get_one_hot(words[j]).to(device)
            x_1 = get_one_hot(words[j+1]).to(device)
            x_2 = get_one_hot(words[j-1]).to(device)
            a.append(net(x,x_1,x_2))        
        for j in range(len(a)):
            f.write(predict_tag(a[j])+" ")
        f.write("\n")
f.close()


<ipython-input-28-e786d0ef8d57>:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final = F.softmax(self.fc2(x_com))


In [50]:
out = open('out.txt').read().split('\n')
error = 0
total = 0
for i in range(len(out)):
    print(i)
    out_tags = out[i].split(' ')
    target_tags = y_test[i].split(' ')
    for j in range(len(target_tags)):
        print(out_tags[j],target_tags[j])
        if(out_tags[j]!=target_tags[j]):
            error+=1
        total+=1
print(error)
print(total)
print((total-error)/total)

0
lwg__psp vmod
nmod__adj lwg__psp
nmod__adj pof__cn
ccof ccof
rsym rsym
nmod__adj pof__cn
ccof ccof
k1 k1
k1 ccof
lwg__psp lwg__rp
nmod__adj nmod__adj
k1 k1s
main main
rsym rsym
 
1
nmod__adj nmod__adj
k1 r6
lwg__psp lwg__psp
k1 k7p
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 k1
main main
rsym rsym
k1 k7p
nmod__adj lwg__rp
nmod__adj r6
lwg__psp lwg__psp
nmod__adj k2
ccof ccof
k1 r6
nmod__adj pof
nmod__adj ccof
lwg__psp lwg__psp
k1 k7t
k1 lwg__rp
k1 pof
main nmod__relc
lwg__vaux lwg__vaux
lwg__vaux lwg__vaux_cont
rsym rsym
 
2
k1 r6
lwg__psp lwg__psp
k1 k1
nmod__adj k7p
lwg__psp lwg__psp
nmod__adj ccof
k1 k7
k1 ccof
lwg__psp lwg__psp
lwg__psp lwg__psp
lwg__psp lwg__psp
ccof k1s
main main
rsym rsym
 
3
nmod__adj k1
k1 nmod
lwg__psp lwg__psp
nmod__adj nmod__adj
nmod__adj ccof
ccof lwg__psp
k1 k7p
nmod__adj nmod
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 ccof
ccof lwg__psp
rsym rsym
ccof nmod
rsym rsym
nmod__adj nmod
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 jjmod__intf
nmod__adj nmod
ccof ccof

75
nmod__adj nmod__adj
k1 r6
lwg__psp lwg__psp
k1 k1
k1 k1u
lwg__psp lwg__psp
lwg__psp lwg__psp
main pof
ccof ccof
k1 main
nmod__adj k1
nmod__adj nmod__adj
k1 jjmod
ccof ccof
k1 k1s
nmod__adj nmod__adj
k1 jjmod
main ccof
main ccof
rsym rsym
 
76
nmod__adj nmod__adj
k1 k1
main k1s
lwg__vaux main
lwg__vaux rs
nmod__adj nmod__adj
k1 k7
lwg__psp lwg__psp
lwg__psp lwg__psp
nmod__adj nmod__adj
nmod__adj nmod__adj
nmod__adj k1
main k1
lwg__vaux lwg__rp
nmod__adj k7p
lwg__psp lwg__psp
nmod__adj nmod__adj
nmod__adj r6
lwg__psp lwg__psp
k1 k2
lwg__psp lwg__psp
lwg__vaux nmod__relc
lwg__vaux lwg__vaux
lwg__vaux lwg__vaux_cont
rsym rsym
lwg__vaux ccof
lwg__vaux lwg__vaux
rsym rsym
 
77
nmod__adj nmod__adj
k1 k7p
lwg__psp lwg__psp
nmod__adj nmod__adj
main k1
lwg__vaux main
rsym rsym
 
78
nmod__adj nmod__adj
k1 k1
nmod__adj mod
rsym rsym
ccof nmod__adj
rsym rsym
k1 k7p
lwg__psp lwg__psp
main k1s
lwg__vaux main
rsym rsym
 
79
nmod__adj nmod__adj
nmod__adj nmod
nmod__adj pof__cn
k1 k1
lwg__psp lwg__ps

lwg__psp lwg__psp
nmod__adj r6
lwg__psp lwg__psp
k1 r6-k2
lwg__psp lwg__psp
nmod__adj pof
ccof nmod__relc
rsym rsym
 
134
k1 r6
lwg__psp lwg__psp
lwg__vaux pof__cn
k1 k7
lwg__psp lwg__psp
k1 pof
k1 nmod
lwg__psp lwg__psp
nmod__adj nmod__adj
nmod__adj nmod__adj
k1 r6
lwg__psp lwg__psp
k1 k1
nmod__adj pof__cn
nmod__adj pof__cn
k1 k1s
lwg__vaux main
rsym rsym
 
135
nmod__adj k1
nmod__adj pof__cn
k1 pof__cn
nmod__adj k7p
lwg__psp lwg__psp
nmod__adj r6
lwg__psp lwg__psp
lwg__psp k1s
lwg__vaux main
rsym rsym
 
136
k1 k1
lwg__psp lwg__psp
k1 k1
main main
rsym rsym
k1 k1
nmod__adj r6
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 r6
lwg__psp lwg__psp
k1 k7
lwg__psp lwg__psp
k1 pof
main k2
lwg__vaux lwg__vaux
lwg__vaux lwg__vaux_cont
nmod__adj k4
nmod__adj nmod__adj
k1 k7p
lwg__psp lwg__psp
lwg__psp nmod
lwg__psp lwg__psp
k1 k7
lwg__psp lwg__psp
k1 r6
lwg__psp lwg__psp
nmod__adj r6
lwg__psp lwg__psp
ccof ccof
k1 nmod
nmod__adj ccof
k1 r6-k2
lwg__psp lwg__psp
nmod__adj pof
k1 r6
lwg__psp lwg__psp
k1 k

lwg__psp lwg__rp
nmod__adj nmod__adj
k1 k7p
lwg__psp lwg__psp
k1 pof
main ccof
lwg__vaux lwg__vaux
lwg__vaux lwg__vaux_cont
rsym rsym
 
163
nmod__adj sent-adv
nmod__adj pof__cn
k1 k1
lwg__psp r6
k1 k5
lwg__psp lwg__psp
main ccof
lwg__vaux lwg__vaux
ccof main
k1 k1
nmod__adj nmod__adj
k1 k7
lwg__psp lwg__psp
nmod__adj adv
lwg__psp lwg__rp
k1 r6
lwg__psp lwg__psp
k1 k2
main ccof
rsym rsym
 
164
nmod__adj nmod__adj
k1 k7t
nmod__adj r6
lwg__psp lwg__psp
k1 nmod
nmod__adj pof__cn
k1 k1
lwg__psp lwg__psp
main main
lwg__vaux k2
nmod__adj k1
ccof ccof
k1 rh
k1 ccof
lwg__psp lwg__psp
lwg__psp lwg__psp
main ccof
rsym rsym
 
165
main main
lwg__vaux lwg__vaux
ccof ccof
k1 k7
nmod__adj ccof
lwg__psp lwg__psp
k1 mod
nmod__adj k1
k1 k1s
lwg__vaux k2
rsym rsym
 
166
k1 main
k1 k2
nmod__adj k7t
nmod__adj r6
lwg__psp lwg__psp
nmod__adj jjmod__intf
nmod__adj nmod__adj
nmod__adj mod
k1 r6
lwg__psp lwg__psp
ccof nmod__relc
k1 k7t
nmod__adj k2
k1 k1
k1 k1s
lwg__vaux ccof
nmod__adj rs
k1 adv
k1 k7t
lwg__psp 

main ccof
lwg__vaux lwg__vaux
main main
nmod__adj r6
k1 ras-k1
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 k1
main ccof
lwg__vaux lwg__vaux
main lwg__vaux_cont
rsym rsym
 
203
nmod__adj r6
lwg__psp lwg__psp
k1 k7
lwg__psp lwg__psp
k1 mod
nmod__adj nmod__adj
k1 r6
lwg__psp lwg__psp
k1 k1
main k1s
lwg__vaux main
rsym rsym
 
204
nmod__adj nmod__adj
nmod__adj pof__cn
nmod__adj k7
lwg__psp lwg__psp
k1 r6
lwg__psp lwg__psp
nmod__adj nmod__adj
nmod__adj nmod__adj
k1 k1
lwg__psp lwg__psp
nmod__adj pof__cn
k1 vmod
lwg__psp lwg__psp
lwg__psp lwg__psp
nmod__adj pof__cn
k1 k2
k1 pof
main main
lwg__vaux lwg__vaux
rsym rsym
 
205
nmod__adj pof__cn
k1 nmod__adj
nmod__adj pof__cn
nmod__adj pof__cn
k1 k1
lwg__psp lwg__psp
k1 k7t
lwg__psp lwg__psp
main main
lwg__vaux k2
nmod__adj nmod__adj
nmod__adj pof__cn
nmod__adj k7
lwg__psp lwg__psp
k1 k1
lwg__psp lwg__psp
nmod__adj pof__cn
k1 k2
lwg__psp lwg__psp
k1 pof
nmod__adj r6
nmod__adj lwg__vaux
lwg__psp lwg__psp
lwg__psp k7t
lwg__psp lwg__psp
lwg__psp lwg__ps

...

rsym k7
lwg__psp lwg__psp
nmod__adj r6
lwg__psp lwg__psp
ccof k2
rsym rsym
k1 nmod
lwg__psp lwg__psp
ccof rs
rsym rsym
main main
lwg__vaux lwg__vaux
rsym rsym
 
1606
nmod__adj nmod__adj
k1 k2
lwg__psp lwg__psp
nmod__adj nmod__adj
lwg__psp k7t
k1 k7
lwg__psp lwg__psp
lwg__psp lwg__psp
k1 k2
lwg__psp rt
lwg__psp lwg__psp
lwg__psp lwg__psp
k1 k2
nmod__adj pof
k1 rt
lwg__psp lwg__psp
k1 pof
main main
lwg__vaux lwg__vaux
lwg__vaux lwg__vaux_cont
rsym rsym
 
1607
k1 r6
k1 k2
main lwg__rp
nmod__adj mod
nmod__adj nmod__adj
k1 k2s
main main
lwg__vaux lwg__vaux
lwg__vaux lwg__vaux_cont
rsym rsym
 
1608
nmod__adj nmod__adj
lwg__psp sent-adv
lwg__psp lwg__psp
nmod__adj pof__cn
ccof ccof
k1 k7p
nmod__adj nmod__adj
nmod__adj nmod__adj
k1 ccof
lwg__psp lwg__psp
nmod__adj nmod__adj
rsym k7
lwg__psp lwg__psp
nmod__adj r6
lwg__psp lwg__psp
nmod__adj k2
main main
lwg__vaux lwg__vaux
rsym rsym
 
1609
nmod__adj nmod__adj
nmod__adj k1
k1 r6
lwg__psp lwg__psp
nmod__adj ccof
lwg__psp lwg__psp
lwg__psp lwg__ps

nmod__adj nmod
nmod__adj pof__cn
k1 pof__cn
ccof ccof
main k1
nmod__adj pof__cn
k1 nmod
k1 pof__cn
k1 ccof
nmod__adj nmod__adj
k1 k7
lwg__psp lwg__psp
ccof k1s
main main
rsym rsym
 
1640
k1 k1
main main
lwg__vaux rs
nmod__adj r6
lwg__psp lwg__psp
nmod__adj nmod
nmod__adj pof__cn
k1 pof__cn
k1 k1
lwg__psp lwg__psp
nmod__adj r6
lwg__psp lwg__rp
lwg__psp lwg__psp
k1 k7
lwg__psp lwg__psp
k1 pof__cn
k1 k2
nmod__adj pof
main ccof
lwg__vaux lwg__vaux
rsym rsym
 
1641
nmod__adj pof
k1 r6
lwg__psp lwg__psp
nmod__adj k1
k1 k7t
lwg__psp lwg__psp
k1 pof
main main
rsym rsym
 
1642
k1 k7t
lwg__psp lwg__psp
k1 r6
nmod__adj nmod__adj
k1 k2
lwg__psp lwg__psp
nmod__adj adv
k1 r6
lwg__psp lwg__psp
k1 k7
lwg__psp lwg__psp
k1 k1
main main
lwg__vaux lwg__vaux
rsym rsym
 
1643
k1 k5
lwg__psp lwg__psp
k1 pof
main nmod__k1inv
nmod__adj k1
k1 r6
lwg__psp lwg__rp
lwg__psp lwg__psp
k1 k7p
lwg__psp lwg__psp
k1 pof
main ccof
lwg__vaux main
nmod__adj r6
lwg__psp lwg__psp
k1 k1
lwg__psp lwg__psp
ccof ccof
k1 k7
nmod_

nmod__adj pof__cn
nmod__adj k1
nmod__adj nmod__adj
k1 ras-k1
lwg__psp lwg__psp
lwg__psp lwg__psp
nmod__adj pof__cn
k1 k7t
lwg__psp lwg__psp
nmod__adj pof
ccof main
rsym rsym
 
1695
k1 r6
k1 k1
main main
nmod__adj k2
k1 k2
nmod__adj pof
k1 k1
lwg__psp lwg__psp
k1 mod
k1 k2
k1 lwg__rp
nmod__adj lwg__neg
ccof ccof
rsym rsym
 
1696
nmod__adj vmod
k1 mod
nmod__adj nmod__adj
k1 k2
k1 lwg__neg
ccof ccof
k1 main
k1 r6-k2
lwg__psp lwg__psp
ccof pof
k1 lwg__rp
nmod__adj ccof
k1 lwg__rp
main lwg__vaux
rsym rsym
 
1697
k1 r6
lwg__psp lwg__psp
nmod__adj k1
nmod__adj nmod__adj
k1 ras-k1
lwg__psp lwg__psp
lwg__psp lwg__psp
lwg__psp pof__cn
k1 k7t
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 rt
lwg__psp lwg__psp
lwg__psp lwg__psp
ccof main
rsym rsym
 
1698
k1 r6
k1 k1
main main
nmod__adj k2
k1 k2
nmod__adj pof
k1 r6
lwg__psp lwg__psp
k1 mod
k1 k2
k1 lwg__rp
nmod__adj lwg__neg
ccof ccof
nmod__adj rh
nmod__adj nmod__adj
k1 k1
k1 k3
lwg__psp lwg__psp
k1 k2
lwg__psp lwg__psp
lwg__vaux ccof
lwg__vaux lwg__vaux

nmod__adj pof__cn
rsym rsym
 k7p
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 vmod
lwg__psp lwg__psp
lwg__psp lwg__psp
k1 k7t
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 k7
lwg__psp lwg__psp
lwg__psp lwg__psp
nmod__adj pof__cn
nmod__adj pof__cn
k1 k1
k1 pof
main main
rsym rsym
 
1757
nmod__adj nmod__adj
k1 r6
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 rt
lwg__psp lwg__psp
lwg__psp lwg__psp
nmod__adj pof__cn
k1 r6
lwg__psp lwg__psp
k1 nmod
nmod__adj pof__cn
k1 k1u
lwg__psp lwg__psp
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 k7
lwg__psp lwg__psp
nmod__adj k1
nmod__adj pof__cn
k1 k2
lwg__psp lwg__psp
k1 vmod
nmod__adj rt
lwg__psp lwg__psp
lwg__psp lwg__psp
main pof
lwg__vaux main
rsym rsym
 
1758
nmod__adj nmod__adj
lwg__psp k7t
nmod__adj pof__cn
rsym rsym
nmod__adj pof__cn
k1 vmod
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 k1
nmod__adj main
ccof lwg__vaux
rsym rsym
 
1759
nmod__adj nmod__adj
k1 k7t
lwg__psp lwg__psp
k1 nmod
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 k1
k1 pof__cn
k1 k7t
lwg__psp lwg__p

k1 k2
k1 pof
main ccof
rsym rsym
 
1793
ccof pof__cn
rsym rsym
 k7t
lwg__psp lwg__psp
nmod__adj r6
lwg__psp lwg__psp
nmod__adj nmod__adj
nmod__adj mod
nmod__adj nmod
nmod__adj pof__cn
nmod__adj pof__cn
k1 k1
nmod__adj nmod__adj
k1 k7p
lwg__psp lwg__psp
nmod__adj k2
ccof rt
lwg__vaux main
lwg__vaux lwg__vaux
rsym rsym
 
1794
nmod__adj k7t
k1 pof__cn
nmod__adj k1
nmod__adj nmod__adj
k1 r6
lwg__psp lwg__psp
ccof k1s
main nmod__relc
lwg__vaux lwg__vaux
lwg__vaux lwg__vaux_cont
rsym rsym
ccof k7t
k1 k1
lwg__psp lwg__rp
nmod__adj pof__cn
k1 r6-k2
lwg__psp lwg__psp
k1 pof
main main
lwg__vaux lwg__vaux
rsym rsym
 
1795
k1 pof__cn
nmod__adj pof__cn
k1 k5
lwg__psp lwg__psp
nmod__adj k2
k1 pof
k1 r6
lwg__psp lwg__psp
nmod__adj nmod__adj
k1 k1
k1 nmod__adj
nmod__adj k7t
ccof k1s
main main
rsym rsym
 
1796
k1 k7t
lwg__psp lwg__psp
nmod__adj pof__cn
nmod__adj k1
k1 pof__cn
k1 r6
lwg__psp lwg__psp
k1 rh
lwg__psp lwg__psp
nmod__adj pof__cn
ccof k2p
main main
lwg__vaux lwg__vaux
rsym rsym
 
1797
lwg__p